In [15]:
import time
import os
from torch.autograd import Variable
import torchvision
import torch
import random
import numpy as np
import numpy
import networks
# from my_args import  args
# from scipy.misc import imread, imsave
from imageio import imread
from PIL import Image

from AverageMeter import  *
import shutil
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

## load model

In [16]:
class Args():
    
    def __init__(self):
        self.filter_size = 4
        self.time_step = 0.5
        self.channels = 3
        self.netName = 'DAIN'
        self.use_cuda = True
        
        self.save_which = 1
        self.dtype = torch.cuda.FloatTensor
        
args = Args()

In [17]:
model = networks.__dict__[args.netName](    channel=args.channels,
                                    filter_size = args.filter_size ,
                                    timestep=args.time_step,
                                    training=False)

In [4]:
if args.use_cuda:
    print('USING CUDA')
    model = model.cuda()

USING CUDA


In [5]:
args.SAVED_MODEL = '../../models/dain/best.pth'

In [6]:
if os.path.exists(args.SAVED_MODEL):
    print("The testing model weight is: " + args.SAVED_MODEL)
    if not args.use_cuda:
        pretrained_dict = torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage)
        # model.load_state_dict(torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage))
    else:
        pretrained_dict = torch.load(args.SAVED_MODEL)
        # model.load_state_dict(torch.load(args.SAVED_MODEL))

    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    # 4. release the pretrained dict for saving memory
    pretrained_dict = []
else:
    print("*****************************************************************")
    print("**** We don't load any trained weights **************************")
    print("*****************************************************************")

The testing model weight is: ../../models/dain/best.pth


In [7]:
model = model.eval()

In [8]:
use_cuda=args.use_cuda
save_which=args.save_which
dtype = args.dtype
unique_id =str(random.randint(0, 100000))

In [9]:
for folder, _, files in os.walk('E:\scriptieAI\code\DAIN\MiddleBurySet\other-data\Walking'):
    frame1 = os.path.join(folder, files[0])
    frame2 = os.path.join(folder, files[1])
    frame3 = os.path.join(folder, files[2])
    


In [10]:
plt.imshow(y_[0].astype(int))

NameError: name 'y_' is not defined

In [11]:
def get_padding(intWidth, intHeight, channel):
    
    if intWidth != ((intWidth >> 7) << 7):
        intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
        intPaddingLeft =int(( intWidth_pad - intWidth)/2)
        intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
    else:
        intWidth_pad = intWidth
        intPaddingLeft = 32
        intPaddingRight= 32

    if intHeight != ((intHeight >> 7) << 7):
        intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
        intPaddingTop = int((intHeight_pad - intHeight) / 2)
        intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
    else:
        intHeight_pad = intHeight
        intPaddingTop = 32
        intPaddingBottom = 32
    
    
    return intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom




def preprocess_image(im):
    X = torch.from_numpy( np.transpose(im, (2,0,1)).astype('float32')/255.0).type(dtype)
    
    intWidth = X.size(2)
    intHeight = X.size(1)
    channel = X.size(0)
    
    
    intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom = get_padding(intWidth, intHeight, channel)
    
    pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom])
    
    torch.set_grad_enabled(False)
    X = Variable(torch.unsqueeze(X,0))
    X = pader(X)
    
    return X, (intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom)
    

    
    
def interpolate(im1, im2):
    X0, pad_0 = preprocess_image(im1)
    X1, pad_1 = preprocess_image(im2)
    
    y_ = torch.FloatTensor()

    assert pad_0 == pad_1
    
    intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom = pad_0
    
    if use_cuda:
        X0 = X0.cuda()
        X1 = X1.cuda()
        
    y_s,offset,filter = model(torch.stack((X0, X1),dim = 0))
    y_ = y_s[save_which]
    
    
    if use_cuda:
        X0 = X0.data.cpu().numpy()
        if not isinstance(y_, list):
            y_ = y_.data.cpu().numpy()
        else:
            y_ = [item.data.cpu().numpy() for item in y_]
        offset = [offset_i.data.cpu().numpy() for offset_i in offset]
        filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
        X1 = X1.data.cpu().numpy()
    else:
        X0 = X0.data.numpy()
        if not isinstance(y_, list):
            y_ = y_.data.numpy()
        else:
            y_ = [item.data.numpy() for item in y_]
        offset = [offset_i.data.numpy() for offset_i in offset]
        filter = [filter_i.data.numpy() for filter_i in filter]
        X1 = X1.data.numpy()
        
    X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))


    y_ = [np.transpose(255.0 * item.clip(0,1.0)[:, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for item in y_]


    offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
    filter = [np.transpose(
        filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
        (1, 2, 0)) for filter_i in filter]  if filter is not None else None
    X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
    
    return y_[0]
    
    
def interpolate_old(im1, im2):
    X0 = torch.from_numpy( np.transpose(im1, (2,0,1)).astype('float32')/255.0).type(dtype)
    X1 = torch.from_numpy( np.transpose(im2, (2,0,1)).astype('float32')/255.0).type(dtype)

    y_ = torch.FloatTensor()

    assert (X0.size(1) == X1.size(1))
    assert (X0.size(2) == X1.size(2))
    
    intWidth = X0.size(2)
    intHeight = X0.size(1)
    channel = X0.size(0)

    if intWidth != ((intWidth >> 7) << 7):
        intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
        intPaddingLeft =int(( intWidth_pad - intWidth)/2)
        intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
    else:
        intWidth_pad = intWidth
        intPaddingLeft = 32
        intPaddingRight= 32

    if intHeight != ((intHeight >> 7) << 7):
        intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
        intPaddingTop = int((intHeight_pad - intHeight) / 2)
        intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
    else:
        intHeight_pad = intHeight
        intPaddingTop = 32
        intPaddingBottom = 32

    pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom])
    
    torch.set_grad_enabled(False)
    X0 = Variable(torch.unsqueeze(X0,0))
    X1 = Variable(torch.unsqueeze(X1,0))
    X0 = pader(X0)
    X1 = pader(X1)
    
    print('mean', X0.mean())
    
    if use_cuda:
        X0 = X0.cuda()
        X1 = X1.cuda()
        
    y_s,offset,filter = model(torch.stack((X0, X1),dim = 0))
    y_ = y_s[save_which]
    
    
    if use_cuda:
        X0 = X0.data.cpu().numpy()
        if not isinstance(y_, list):
            y_ = y_.data.cpu().numpy()
        else:
            y_ = [item.data.cpu().numpy() for item in y_]
        offset = [offset_i.data.cpu().numpy() for offset_i in offset]
        filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
        X1 = X1.data.cpu().numpy()
    else:
        X0 = X0.data.numpy()
        if not isinstance(y_, list):
            y_ = y_.data.numpy()
        else:
            y_ = [item.data.numpy() for item in y_]
        offset = [offset_i.data.numpy() for offset_i in offset]
        filter = [filter_i.data.numpy() for filter_i in filter]
        X1 = X1.data.numpy()
        
    X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))


    y_ = [np.transpose(255.0 * item.clip(0,1.0)[:, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for item in y_]
    # 0 weggehaald als index

    offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
    filter = [np.transpose(
        filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
        (1, 2, 0)) for filter_i in filter]  if filter is not None else None
    X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
    del X0; del X1
    return y_[0]


In [14]:
# load video

filepath_video = 'E:/scriptieAI/rai_slide_lq.mp4'

cap = cv2.VideoCapture(filepath_video)

cap.isOpened(), cap.get(cv2.CAP_PROP_FRAME_COUNT)


(True, 281.0)

In [15]:
# interate through frames
frames = []

n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('n frames', n_frames)
for i, frame in tqdm(enumerate(range(n_frames))):
    ret, img = cap.read()
#     print(i, img.shape)

    frames.append(img)

    
    if ret == False:
        break
    
    

n frames 281


281it [00:00, 1047.40it/s]


In [16]:
frame.shape

AttributeError: 'int' object has no attribute 'shape'

In [17]:
frame_a.shape

NameError: name 'frame_a' is not defined

In [18]:
i = 0
frame_a = frames[i]
frame_b = frames[i+1]

Xi = interpolate_old(frame_a, frame_b)

mean tensor(0., device='cuda:0')


c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and af

## check stochasticity

In [19]:
result1 = interpolate_old(frame_a, frame_b)
result2 = interpolate_old(frame_a, frame_b)

mean tensor(0., device='cuda:0')


c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and af

mean tensor(0., device='cuda:0')


..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd fu

In [21]:
(result1 - result2).min()

0.0

In [15]:

interpolated = []


for i in tqdm(range(len(frames[1:]))):
    frame_a = frames[i]
    frame_b = frames[i+1]
    
    Xi = interpolate_old(frame_a, frame_b)
    interpolated.append(Xi)
    
    
    

  0%|                                                                                          | 0/280 [00:00<?, ?it/s]c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_

In [15]:

interpolated = []


for i in tqdm(range(len(frames[1:]))):
    frame_a = frames[i]
    frame_b = frames[i+1]
    
    Xi = interpolate_old(frame_a, frame_b)
    interpolated.append(Xi)
    
    
    

  0%|                                                                                          | 0/280 [00:00<?, ?it/s]c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_

In [127]:
img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [16]:
frames[0].shape

(480, 640, 3)

In [18]:
frames.__len__()

281

561

In [23]:
for i, inter in tqdm(enumerate(interpolated)):
    frames.insert((i*2)+1, inter)

280it [00:00, ?it/s]


In [30]:
# write files to temp folder
# out = cv2.VideoWriter('testvideo.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (480, 640))

# for frame in frames:
#     out.write(frame)

# out.release()

In [36]:
frames = np.stack(frames)

In [37]:
torchvision.io.write_video('testvideo.mp4', torch.from_numpy(frames), 30)

In [53]:
torch.zeros(3,42).nelement()

126

In [57]:
t = 0
for element in model.parameters():
    t += element.nelement()

In [58]:
t

24031656

In [ ]:
# put together to video

# audio?

In [55]:
import pandas as pd

In [41]:
with open('../../results.pkl', 'rb') as f:
    r = pickle.load(f)

In [56]:
pd.DataFrame(r)

,DAIN,SepConv,QVI
ie,"[15.34152889251709, 15.34152889251709, 15.9646...",[],[]
psnr,"[72.54423522949219, 72.54423522949219, 72.1984...",[],[]
ssim,"[0.9839091300964355, 0.9839091300964355, 0.981...",[],[]
